### **6.5 - [Practica] Make your first conditional task using branching**

En este vídeo vamos a ver cómo ejecutar diferentes tareas en función de un valor dado. Desde tu editor de código, comprueba que te encuentras en la carpeta airflow-materials/airflow-section-6 y abre el archivo branch_dag.py en mnt/airflow/dags.  

<center><img src="https://i.postimg.cc/d0BHCtFB/a951.png"></center>
<center><img src="https://i.postimg.cc/FHXDnJsp/a952.png"></center>

Este DAG se encarga de solicitar diferentes APIs para geolocalizar direcciones ip.  Aquí se realiza una comprobación desde la función "check_api", donde se realiza una petición para cada API y si la petición devuelve datos JSON (JSON data) y que tengan el campo "country", entonces se devuelve la API.  Este valor devuelto es procesado a continuación por el BranchPythonOperator para ejecutar la tarea correspondiente.  Por ejemplo, si el valor devuelto es "ipstack", se ejecutará la tarea "ipstack" y se omitirán las demás tareas.  Si no hay ninguna API disponible, la función check_api devuelve "none" y, por tanto, sólo se ejecutará la tarea "none".  Finalmente se crean las tareas correspondientes a las APIs a partir de este "bucle for" así como las depencias con la tarea branch y la tarea store.  Bien, sin más esperas vamos a ver lo que obtenemos de la interfaz de usuario de Airflow.  Pero primero, abre tu terminal y comprueba que estás en la carpeta airflow-materials/airflow-section-6 y luego escribe el comando "docker-compose -f docker-compose-CeleryExecutor.yml up -d". Enter. Ok Podemos comprobar que todo se ha iniciado como se esperaba con el comando "docker ps".  Perfecto. Ahora, abre tu navegador web, y ve a localhost:8080.  Enter. Pincha en "branch_dag" y en "Graph View". 

<center><img src="https://i.postimg.cc/BbdM3j5q/a953.png"></center>

Como puedes ver, varias tareas son downstream de la tarea check_api donde cada tarea tiene el nombre de su correspondiente API, así como la tarea none. La última tarea del DAG es "save" y debe ejecutarse siempre.  Muy bien, vamos a disparar el DAG para ver qué pasa.  Haga clic en el toggle aquí para programar el DAG y actualice la página hasta que el proceso haya terminado.  Bien, el DAGRun es un éxito como se muestra aquí, por lo que ha terminado.  

<center><img src="https://i.postimg.cc/VsT7nVx3/a954.png"></center>

Como puede observar, solo la tarea "ip-api" ha sido ejecutada y las otras tres tareas han sido omitidas como se muestra por los colores de sus bordes.  Si hace clic en la tarea "check_api", luego en "view log". En la parte inferior de los logs, tenemos el valor "ip-api" dado el id de la tarea de la siguiente tarea a ejecutar. 

<center><img src="https://i.postimg.cc/9QPJ3FDh/a955.png"></center>
<center><img src="https://i.postimg.cc/LsHvmKPZ/a956.png"></center>

La línea justo debajo indica que el branch a seguir comienza con la tarea ip-api.  Finalmente, aquí tenemos los ids de las tres tareas saltadas por el BranchPythonOperator.  

<center><img src="https://i.postimg.cc/C15NfGmh/a957.png"></center>

Ok hagamos una modificación rápida para ejecutar "ipinfo" por ejemplo. Vuelve a tu editor de código, en la función "check_api", cambia la línea "return api" por "return ipinfo".  Ahora, la función siempre devolverá ipinfo y así el BranchPythonOperator siempre elegirá la tarea "ipinfo". Guarde el archivo y vuelva a la interfaz de usuario de Airflow.  

<center><img src="https://i.postimg.cc/MGd5TP03/a958.png"></center>

Consulta el DAG haciendo clic en este pequeño botón para que se apliquen las modificaciones.  Ok. Haga clic en "Graph View" y dispare el DAG manualmente haciendo clic aquí.  "Ok".  Vuelve a "Graph View" y empieza a refrescar la página hasta que se termine el DAGRun.  Muy bien, como se esperaba, ahora la tarea "ipinfo" se ha disparado y las otras se han saltado.  

<center><img src="https://i.postimg.cc/zXgxjk53/a959.png"></center>

De nuevo, si echamos un vistazo a los logs de la tarea "check_api", el valor devuelto es "ipinfo" y las tareas saltadas son las siguientes.  

<center><img src="https://i.postimg.cc/gj6HgGC1/a960.png"></center>

Así es como se puede elegir entre diferentes tareas en función de un valor dado.  En realidad, este ejemplo es bastante simple.  El valor podría haberse dado desde una base de datos o de cualquier otra forma que se te ocurra.  Si vuelves al "Graph View", para los ojos más avispados, te habrás dado cuenta de que la última tarea "save" se ha saltado también lo cual no es algo que queramos.  Esto está relacionado con el tema que descubrirá en el siguiente vídeo sobre las trigger rules. Por defecto, una tarea se dispara sólo si sus padres han tenido éxito. Aquí, debido a que algunas tareas son omitidas, no son consideradas como exitosas y por lo tanto, la "trigger rule" por defecto para la tarea "save" no es respetada.  Por eso la tarea no se ejecuta.  ¿Cómo podemos solucionarlo?  Bien, desde su editor de código, desplácese hacia abajo hasta llegar a la tarea "save" y añada el parámetro "trigger_rule=one_success".  

<center><img src="https://i.postimg.cc/bdcHMK8r/a961.png"></center>

Por defecto el valor es all_success, lo que significa que todos los padres deben haber tenido éxito para activar esa tarea. Al cambiar este valor a "one_success" significa que, si al menos uno de los padres tuvo éxito, entonces la tarea puede ser disparada. Guarde el archivo y vuelva a la interfaz de usuario. Refresca el DAG haciendo click en este pequeño botón (refresh) y ahora que las modificaciones han sido aplicadas puedes volver a "Graph view", activa el DAG haciendo click aquí y comienza a refrescar la página.  Ahora espera a que termine el DagRun.  Ok, El DAGRun está hecho.

<center><img src="https://i.postimg.cc/9MkBPDnG/a962.png"></center>

Si echamos un vistazo al DAG, podemos ver que esta vez la tarea "save" se ha disparado como se esperaba.  Bien, lo último que me gustaría mostrarte es que puedes devolver múltiples ids de tareas para elegir varios branches a la vez.  Hagamos algunas modificaciones en el DAG que utilizamos.  De vuelta a tu editor de código, desde la función check_api, en lugar de devolver la primera API disponible, vamos a devolver una lista con todas las APIs a las que podemos acceder. Para ello, creamos aquí una variable de lista vacía llamada "apis".  Luego cambiamos la instrucción "return api" por "apis.append (api)".  Así, cada vez que la API esté disponible, se añadirá a la lista de APIs.  Finalmente, si la lista contiene al menos una api, la devolvemos, de lo contrario, devolvemos el task_id "none".  Así.  Muy bien, guarde el archivo y vaya a la interfaz de usuario de Airflow.  

<center><img src="https://i.postimg.cc/y6wMsx8g/a963.png"></center>

Desde allí refrescamos el dag haciendo click en este botón y lo disparamos haciendo click aquí, luego empezamos a refrescar la página hasta que el DAGRun termine.  Ok, si pulsamos sobre "Graph View", podemos ver que ahora, se han ejecutado dos tareas en lugar de sólo una como antes.  

<center><img src="https://i.postimg.cc/7PCFF0wr/a964.png"></center>

Así es como se pueden ejecutar múltiples branches usando el BranchPythonOperator.  Bien.  Ahora abre tu terminal y escribe el siguiente comando para detener los contenedores docker que ejecutan Airflow "docker-compose - f docker-compose-CeleryExecutor.yml down".  Enter. Muy bien, espero que hayas disfrutado descubriendo cómo crear diferentes branches en tu DAG, vamos a tomar un breve descanso y nos vemos en el próximo vídeo.  